# How Do We Learn Networks?

**(Run this cell to define useful Latex macros)**
\\[
\newcommand{\bigoh}[1]{\mathcal{O}\left(#1\right)}
\newcommand{\card}[1]{\left\lvert#1\right\rvert}
\newcommand{\condbar}[0]{\,\big|\,}
\newcommand{\eprob}[1]{\widehat{\text{Pr}}\left[#1\right]}
\newcommand{\norm}[1]{\left\lvert\left\lvert#1\right\rvert\right\rvert}
\newcommand{\prob}[1]{\text{Pr}\left[#1\right]}
\newcommand{\pprob}[2]{\text{Pr}_{#1}\left[#2\right]}
\newcommand{\set}[1]{\left\{#1\right\}}
\newcommand{\fpartial}[2]{\frac{\partial #1}{\partial #2}}
\\]

### The Problem: Learning Boolean Functions

I've shown you how any Boolean function can be represented by a network of LR models (with the logistic function replaced by the Heaviside step function).

What I haven't told you is how to *learn* a network of LR models. Here's what I mean.

Consider a boolean function $f: B^m \to B^n$. I give you training examples for some of the $2^m$ possible input settings, and the corresponding output. I want you to find a network of LR models that correspond to this function.

There are four things to be learned:

1. How many layers of gates to use. The *depth* of the network.
2. How many gates to use in each layer. The *width* of each layer of the network.
3. Which gates to use in each layer.
4. How the gates should be wired up.

The depth of the network and the width of each layer is called the *architecture* of the network. The machine learning practitioner typically picks these values. They are not typically learned by the ML algorithm.

Which gates to use in each layer, and how they are wired together *are* learned from the training dataset. These choices represent the parameters of the network. I will show one stupid way to do that first.

### Misclassification Error

First, before we can decide how to learn from the data, we need a notion of "goodness of network."

The simplest possibility is that we calculate the percentage of training examples that the network gets the right output on. Higher is better. This is called the *misclassification error*.

An important note: our network, for now, is *nonprobabilistic.* It doesn't calculate probabilities that an output should be $True$ or $False$, it just outputs its guess of $True$ or $False$. Therefore we can't use our cross entropy error yet, which uses continuous valued probabilities.


### Hill Climbing

Start with a random setting of the gates to use and how they are wired together.

For lack of any better idea, let's have an iterative where the algorithm randomly chooses a gate to change. It randomly chooses a new candidate to switch it to. It will randomly choose either one (if a NOT gate) or two (if an AND or OR gate) inputs from the prior layer. It will leave the connection the same.

If this would improve the misclassification error, it makes the change. Otherwise it doesn't make the proposed change. This is called *hill climbing*.

You run this for a long time, until the network is no longer able to make any more positive improvements.

Of course, the network may find a *local optimum*. It may be that no small tweak to the current network can make things better, but that there is a very different network that is better. Because of this possibility, you might want to run several iterations of the algorithm, choosing different starting configurations. You can keep track of the best that you find.

This algorithm is stupid and inefficient. The problem is that there is no intelligence or principle behind the proposed changes: they are random. Also, only one parameter is being changed at a time. This is a waste because evaluating a proposed changed network is expensive (requires looking at all the training data).

You can try to change more parameters in each step, but the problem is that this is more different than your currently best known parameters. Think about it: if at each step we tried a totally new random network, we wouldn't be iteratively improving on our solution at all.


### Error Derivatives Gives Direction

In the case of linear and logistic regression, the error derivative showed us in what direction to change each of the parameters. There was also an efficient, *vectorized* calculation of these derivatives. Many changes were made *simultaneously*.

It is true that sometimes a proposed GD change can cause the error to go up, because we make many simultaneous changes. Each change independently should be positive, but together they might have a negative impact. However, if the learning rate is low, there is less change of this happening. And regardless, the benefit of changing many parameters simultaneously is worth this risk.


So why not use Gradient Descent to learn our network?

The answer is that we *want* to. However, as currently formulated, our network is not *differentiable*.

First, there are only a discrete set of gates to choose from. The AND, OR, and NOT gates do not form a continuous space of gates. We presently cannot say "what if I made an infinitesimal change to this AND gate to make it a little bit more like OR. Would that improve the error?" We'll have to fix this somehow.

Second, when choosing which gates to wire to which others, this is also a discrete choice. We'll end up fixing this as part of our solution to the first problem.



The fix for the first two problems is simple. Wire every gate to *all* of the gates in the prior layer. Instead of forcing our gate to be just AND/OR/NOT, let the gate apply a weighted sum of the inputs before running it through the Heaviside function.

We can still represent AND/OR/NOT, because those are all linearly separable. But we can also represent new kinds of Boolean functions, too. We can represent any Boolean function where the inputs that correspond to positive and to negative outputs are linearly separable.

Since the weights of a linear function are continuous real values, you can now have a continuous space of linear functions.


The third problem is related to the first two. We can now change the weights continuously, but a small change in the weights will induce *no* change after we run the output through the Heaviside step function. That's because the Heaviside step function has zero derivative everywhere (except at $z = 0$, where it has no derivative).

Let me explain. Say that $z = \theta_0 + \sum_i \theta_i x_i$ is $5.0$ for some example. That means that $H(z)= 1.0$. What would happen to $H(z)$ if we tweaked $z$ by an infinitesimal amount $\partial z$? Nothing! Because the new $z$ value would still be greater than zero, so the new $H(z)$ value would still be 1.0.

We need to put a continuous valued version of the Heaviside function in. The most natural choice is to go back to using the logistic sigmoid function. The "downside" to doing so is that the outputs are now not necessarily exactly zero or one, so the output is not exactly a Boolean value.

On the plus side, we can think of the logistic sigmoid function as a natural continuous, differentiable analogue to the Heaviside step function.


Since our "gates" are now continuous valued, maybe we should stop calling them "gates." Let's call them *units* instead. That is the common neural network terminology.

The logistic function in this context is called an *activation function*. It's what we run the weighted sum through to get the final output of the unit. The input to the activation function is called the *preactivation* (at least by me!). It is common to write the preactivation as $z$. The output is called the *activation* of the unit.

There are other choices of activation function. $tanh$ is a common choice (values $(-1, +1)$). The most common nowadays is the *rectified linear unit*, which outputs $max(z, 0)$. I will talk about these choices some other time.

### Logistic Units Approximate Boolean Gates

I want to note that any Boolean gate where the positive and negative inputs are linearly separable can still be *approximated* by a unit with a logistic function.

Consider a line that linearly separates the positive and negative inputs. There is a logistic unit with this decision boundary (you just have to set the $\theta$ weights right).

Now, close to the decision boundary, $z = \theta_0 + \sum_i \theta_i x_i \approx 0.0$. Therefore, we have that $\sigma(z) \approx 0.50$. This is as non-Boolean as you can get!

If you don't like that, reset the $\theta$ to be $100\theta$. This gives you the same decision boundary. However, the $z$ value now grows $100$ times faster as you move away from the decision boundary. Hopefully $z$ is now large enough (positive or negative) so that $\sigma(z)$ is close to one or zero.

Of course, this just changes the notion of "close to the decision boundary" by a factor of 100. But the point is that for practical matters, you could always shrink "close to the decision boundary" to mean so close as to be unimportant/negligable.

Note that in practice you probably don't want this to happen. But the point remains: any Boolean Network can be approximated to an arbitrary degree by a Neural Network.

### Make Nonlinearity Differentiable

We were previously "cleaning up" the output of a gate by using a function that mapped every negative value to zero, and every positive value to one. This is called the Heaviside Step Function sometimes.

The step function is undesirable. First, it is discontinuous, and doesn't have a derivative at $x = 0$. But that isn't the main problem.

The real problem is that the Heaviside function has a zero derivative everywhere. Let me explain why this is a problem.

Say we are considering an example. For this example, a unit is output zero. Let us say that it would be better if the unit were to output a one.

The way to make this happen is to tweak the parameters for this unit so that its overall input becomes positive.

However, $\fpartial{H}{\theta_{i, j}} = 0$, because the derivative of the Heaviside function is *always* zero. As far as the Heaviside function is concerned, there is no benefit to an incremental improvement to the input if it doesn't switch the function from zero to one.

Since derivatives are always about *marginal* changes, which are infintesimal, it will never realize that changing the parameters could help.


### The Output Is Now Continuous Valued

Now that we are applying the logistic function to the output of each unit, all the outputs of the units are continuous valued in the range of zero to one.

That means that the overall output of the network is also continuous valued, not just zero or one.

Now, we could still apply the Heaviside step function at the end to get a value of zero or one, so that we can use the misclassification error. But that would introduce non-differentiability into the network.

Instead, let's just use the ordinary logistic function at the end, and use the cross entropy error like we used for logistic regression.


In addition to being differentiable, unlike the misclassification error, the cross entropy error fits in with an interpretation of the final output as a probability that the result should be a one.

This is "richer" information than just a binary guess of what the output should be. For instance, if the network is deciding whether we will make money on some stock trade, we can decide to only make the trade if the probability of it being profitable is greater than 75% (or whatever).

Unlike the classification error, the cross entropy error gives credit for approximating this conditional probability distribution properly.

### The Power of Neural Networks

We are done for now! The model is now fully differentiable with respect to the parameters. Now we can use our typical gradient descent approach to train it.

This is what a neural network is! Neural networks are as powerful as Boolean networks, which can represent any Boolean function (effectively: anything a computer can compute).

Moreover, because the units of a neural network can approximate AND and OR gates, they can use units to "refeaturize" the inputs. Each layer of the neural network can be seen as a "refeaturization" of the previous one, so that the final layer, which contains a single unit, can answer the original question with a linear classifier.

The powerful thing is that the network decides what it wants the units to stand for. Instead of us adding in a bunch of new terms $X_{i, j} = X_i X_j$, the network can decide if it wants to use a unit to represent that feature.

Before we had a dilemma: do we know from prior knowledge what features are worth adding in? If not, should we add in a bunch of new features without knowing whether they will help at all?

The power of the neural network is this: we give the network extra capacity via more layers or more units, and *it decides* how to use the new capacity.
